In [1]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=2048,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [3]:
client = InferenceClient(
    ""
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [4]:
topics = [
    'nasi lemak',
    'nasi goreng',
    'teh tarik',
    'teh ais',
    'kopi ais',
    'laksa',
    'kuih muih',
    'sate',
    'roti canai',
    'ewallet',
    'amerika syarikat',
    'rusia',
    'china',
    'afrika',
    'singapore',
    'thailand',
    'eropah',
    'australia',
    'turki',
]

prompts = []
for t in topics:
    prompt = f'generate complex malay questions related to {t} in malaysian context'
    prompts.extend([prompt] * 50)
    
len(prompts)

950

In [5]:
!mkdir mixtral-hubungan-v2

In [6]:
def answer(q, i):
    filename = f'mixtral-hubungan-v2/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [7]:
max_worker = 50

questions = prompts
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

100%|███████████████████████████████████████████| 19/19 [13:12<00:00, 41.72s/it]
